<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

**Importar arquivos de configuração do Kaglle**

In [43]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [ ]:
!kaggle datasets download tanyadayanand/geological-image-similarity
! unzip -qq geological-image-similarity

 51% 17.0M/33.6M [00:00<00:00, 46.3MB/s]
100% 33.6M/33.6M [00:00<00:00, 96.6MB/s]


### Data Preprocessing

In [2]:
andesite_dir = glob.glob('geological_similarity/andesite/*.jpg');gneiss_dir = glob.glob('geological_similarity/gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(schist_dir))


5000
5000
4998
5000
5000


In [ ]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]


In [10]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite',
               'schist']


In [12]:
for i in andesite_dir[0:500]:
    andesite_dir.append([i,'class_names[0]'])
for j in gneiss_dir[0:500]:
   gneiss_dir.append([j,'class_names[1]'])
df = andesite_dir + gneiss_dir
random.shuffle(df)
df

['geological_similarity/andesite/LJ066.jpg',
 ['geological_similarity/gneiss/O29RU.jpg', 'class_names[1]'],
 'geological_similarity/gneiss/H6K5P.jpg',
 'geological_similarity/gneiss/ZIRG5.jpg',
 'geological_similarity/andesite/KBXI3.jpg',
 'geological_similarity/andesite/ME3TZ.jpg',
 ['geological_similarity/gneiss/P0JCL.jpg', 'class_names[1]'],
 'geological_similarity/gneiss/IZKZU.jpg',
 ['geological_similarity/andesite/17P5L.jpg', 'class_names[0]'],
 ['geological_similarity/gneiss/5UP8S.jpg', 'class_names[1]'],
 'geological_similarity/gneiss/C4BYU.jpg',
 'geological_similarity/andesite/1N3PA.jpg',
 'geological_similarity/gneiss/BSYML.jpg',
 'geological_similarity/gneiss/2BRP9.jpg',
 'geological_similarity/andesite/34GGL.jpg',
 ['geological_similarity/gneiss/VL9WN.jpg', 'class_names[1]'],
 'geological_similarity/andesite/HA8SL.jpg',
 'geological_similarity/gneiss/0R7FJ.jpg',
 ['geological_similarity/gneiss/9LSUV.jpg', 'class_names[1]'],
 'geological_similarity/andesite/9ZMSZ.jpg',
 'ge

In [9]:
data_df = pd.DataFrame(df,columns=['PATH','LABEL'])
data_df

ValueError: ignored